In [2]:
import TreeKernel as tk
import TreeBuild as tb
import numpy as np
import pandas as pd

In [3]:
def read_data(path_to_file):
    df = pd.read_csv(path_to_file)
    print ("Shape of base training File = ", df.shape)
    # Remove missing values and duplicates from training data
    df.drop_duplicates(inplace=True)
    df.dropna(inplace=True)
    print("Shape of base training data after cleaning = ", df.shape)
    return df

df_train = read_data("input/train.csv")

Shape of base training File =  (404290, 6)
Shape of base training data after cleaning =  (404288, 6)


In [ ]:
## Establish lambda values
lambdas = [1, 0.8, 0.4, 0.2, 0.1, 0.05]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

df_train['cos_sim'] = df_train.apply(lambda row: gen_cosine_sim(row['question1'],
                                                                    row['question2'],
                                                                    vectorizer), axis=1)
print("Tree Kernel Features Generated")

scaler = MinMaxScaler().fit(df_train[['cos_sim']])
X = scaler.transform(df_train[['cos_sim']])

y = df_train['is_duplicate']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=37)

X_train.shape, X_test.shape, y_train.shape, y_test.shape